# Importación de librerías

In [1]:
import sys
sys.path.append('../')

from src.func_aux_cities import *

# Carga de datos

## Capitales

In [2]:
df = get_df('https://en.wikipedia.org/wiki/List_of_capital_cities_by_elevation')
df.head()

,Country,Capital,Elevation (m),Elevation (ft)
0,Bolivia\n,La Paz\n,"3,640","11,942\n"
1,Ecuador\n,Quito\n,"2,850","9,350\n"
2,Colombia\n,Bogotá\n,"2,625","8,612\n"
3,Ethiopia\n,Addis Ababa\n,"2,355","7,726\n"
4,Bhutan\n,Thimphu\n,"2,334","7,657\n"


In [3]:
df.Country = df.Country.apply(lambda x:x.split('\n')[0])
df.Capital = df.Capital.apply(lambda x:x.split('\n')[0])
df.drop('Elevation (ft)', axis=1, inplace=True)
df.head()

,Country,Capital,Elevation (m)
0,Bolivia,La Paz,"3,640"
1,Ecuador,Quito,"2,850"
2,Colombia,Bogotá,"2,625"
3,Ethiopia,Addis Ababa,"2,355"
4,Bhutan,Thimphu,"2,334"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country        195 non-null    object
 1   Capital        195 non-null    object
 2   Elevation (m)  195 non-null    object
dtypes: object(3)
memory usage: 4.7+ KB


In [5]:
df.columns = ['Country', 'Capital', 'Elevation']
df.Elevation = df.Elevation.apply(lambda x:x.replace(',', ''))
df.Elevation = df.Elevation.apply(lambda x:x.split(' ')[0])
df.Elevation = df.Elevation.astype(int)
df.head()

,Country,Capital,Elevation
0,Bolivia,La Paz,3640
1,Ecuador,Quito,2850
2,Colombia,Bogotá,2625
3,Ethiopia,Addis Ababa,2355
4,Bhutan,Thimphu,2334


## Otras ciudades

In [6]:
df2 = get_df2('https://en.wikipedia.org/wiki/List_of_cities_by_elevation')

In [7]:
df2.columns = ['Country', 'City', 'Continental', 'Latitude', 'Longitude', 'Population', 'Elevation']
df2.head()

,Country,City,Continental,Latitude,Longitude,Population,Elevation
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,"523,000",822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,"747,431","1,395"
2,China,Shanghai,Asia,N31.2304,E121.4737,"26,320,000",122
3,Nepal,Butwal,Asia,N27.6866,E83.4323,"120,982",150
4,Italy,Milan,Europe,N45.4625,E9.186389,"1,378,689",122


In [8]:
df2.dropna(inplace = True)
df2.reset_index()
df2.head()

,Country,City,Continental,Latitude,Longitude,Population,Elevation
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,"523,000",822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,"747,431","1,395"
2,China,Shanghai,Asia,N31.2304,E121.4737,"26,320,000",122
3,Nepal,Butwal,Asia,N27.6866,E83.4323,"120,982",150
4,Italy,Milan,Europe,N45.4625,E9.186389,"1,378,689",122


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 0 to 198
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      61 non-null     object
 1   City         61 non-null     object
 2   Continental  61 non-null     object
 3   Latitude     61 non-null     object
 4   Longitude    61 non-null     object
 5   Population   61 non-null     object
 6   Elevation    61 non-null     object
dtypes: object(7)
memory usage: 3.8+ KB


In [15]:
df2.Population = df2.Population.apply(lambda x:x.replace(',', ''))
df2.Elevation = df2.Elevation.apply(lambda x:x.replace(',', ''))
df2.head()

,Country,City,Continental,Latitude,Longitude,Population,Elevation
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,523000,822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,747431,1395
2,China,Shanghai,Asia,N31.2304,E121.4737,26320000,122
3,Nepal,Butwal,Asia,N27.6866,E83.4323,120982,150
4,Italy,Milan,Europe,N45.4625,E9.186389,1378689,122


In [24]:
a = df2[df2.Elevation=='']
df2.drop(65, inplace=True, axis=0)

In [27]:
df2.reset_index()
df2.head()

,Country,City,Continental,Latitude,Longitude,Population,Elevation
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,523000,822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,747431,1395
2,China,Shanghai,Asia,N31.2304,E121.4737,26320000,122
3,Nepal,Butwal,Asia,N27.6866,E83.4323,120982,150
4,Italy,Milan,Europe,N45.4625,E9.186389,1378689,122


In [29]:
df2.Elevation = df2.Elevation.apply(lambda x:x.split(' ')[0]).astype(int)
df2.Population = df2.Population.astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 198
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      60 non-null     object
 1   City         60 non-null     object
 2   Continental  60 non-null     object
 3   Latitude     60 non-null     object
 4   Longitude    60 non-null     object
 5   Population   60 non-null     int64 
 6   Elevation    60 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 3.8+ KB


In [30]:
# mirar cómo concatenar ambos dataframes